# AI Applications Project: Report Generation

In [ ]:
#Importing the generative ai
%pip install -U -q "google-genai>=1.4.0"

### Setting up API Key

In [ ]:
from google import genai

GOOGLE_API_KEY = input("Enter your Google API key:")
client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
MODEL_ID = "gemini-2.5-flash" # @param ["gemini-2.5-flash-lite","gemini-2.5-flash","gemini-2.5-pro","gemini-2.0-flash"] {"allow-input":true, isTemplate: true}

# Data

## Database Integration

In [ ]:
import numpy as np
import pandas as pd
import os
import sys

sys.path.append('../../')

from Database.db import SessionLocal
from Database_Table import Inventory, Order

def getDbContent():
    session = SessionLocal()
    inventory_records = session.query(Inventory).all()
    order_records = session.query(Order).all()
    session.close()
    return inventory_records, order_records

inventory, order = getDbContent()

In [ ]:
def dbtoList(records):
    output_list = []
    for r in records:
        if isinstance(r, Inventory):
            data = {
                "ItemId": r.ItemId, 
                "ItemName": r.ItemName,
                "Category": r.ItemCategory,
                "Quantity": r.ItemQuantity, 
                "UnitsSold": r.UnitsSold,
                "Weight": r.Weight, 
                "Size": r.Size,
                "Priority": r.Priority, 
                "Location": r.Location,
                "Date": r.Date, 
                "Dispose": r.Dispose                
            }
        elif isinstance(r, Order):
            data = {
                "OrderId": r.OrderId,
                "ItemId": r.ItemId, 
                "OrderQuantity": r.OrderQuantity, 
                "Sales": r.Sales, 
                "Price": r.Price, 
                "Discount": r.Discount,
                "Profit": r.Profit, 
                "DateOrdered": r.DateOrdered,
                "DateReceived": r.DateReceived,
                "CustomerSegment": r.CustomerSegment
            }
        else:
            continue
        output_list.append(data)
    
    return output_list

inventoryData = dbtoList(inventory)
orderData = dbtoList(order)

## Supervised Models

In [ ]:
import pickle
import joblib
from datetime import datetime
from IPython.display import Markdown

class Supervised_Models:
    # Location Prediction Model
    def predict_location(input_data):
        with open('../../Supervised_models/Samuel/storage_prediction_model.pkl', 'rb') as file:
            storage_prediction_model = pickle.load(file)
            
        categorical_features = {
            'Priority': ['High','Low','Medium'],
            'Product_Type': ['Clothing','Technology','Other','Sports and Fitness'],
            'Size': ['Large','Medium','Small']
        }
        numerical_features = ['Order_Quantity', 'Weight']
        one_hot_columns = []
        
        for feature, values in categorical_features.items():
            for value in values:
                one_hot_columns.append(f"{feature}_{value}")
            
        # Combine with numerical features to get all feature names
        all_feature_names = one_hot_columns + numerical_features

        features_dict = {col: 0 for col in all_feature_names}
    
        # Set one-hot encoded features
        for feature, values in categorical_features.items():
            if feature in input_data:
                selected_value = input_data[feature]
                one_hot_col = f"{feature}_{selected_value}"
                if one_hot_col in features_dict:
                    features_dict[one_hot_col] = 1
    
        # Set numerical features
        for feature in numerical_features:
            if feature in input_data:
                features_dict[feature] = float(input_data[feature])
        
        # Convert to array in the correct order
        features_array = np.array([features_dict[col] for col in all_feature_names]).reshape(1, -1)

        prediction = storage_prediction_model.predict(features_array)
        return prediction

    def demand_forecast_preprocessor(order_data, inventory_data):
        # Create Dataframe
        order = pd.DataFrame(order_data)
        inventory = pd.DataFrame(inventory_data)

        # Ensure dates are in datetime format
        order['DateOrdered'] = pd.to_datetime(order['DateOrdered'])
        
        # Extract Month (period or string, depending on preference)
        order['OrderMonth'] = order['DateOrdered'].dt.to_period('M')
        
        # Merge with Category lookup table
        merged_df = order.merge(inventory, on='ItemId', how='left')
        
        # Group and aggregate
        result_df = (
            merged_df
            .groupby(['OrderMonth', 'Category', 'CustomerSegment'], as_index=False)
            .agg(
                AveragePrice=('Price', 'mean'),
                AverageDiscount=('Discount', 'mean')
            )
        )
        
        return result_df

    # Demand forecast model
    def predict_demand_forecast(input_data):
        demand_forecast_model = joblib.load('../../Supervised_models/ShernFai/model/salesforecast(categories).pkl')
        with open('../../Supervised_models/ShernFai/model/salesforecast_preprocessor.pkl', 'rb') as f:
            preprocessor_data = pickle.load(f)

        categories = {
            "Clothing" : [
                "Cleats",
                "Men's Footwear",
                "Women's Apparel"
            ],
            "Technology": [
                "Electronics",
                "Video Games",
                "Cameras",
                "Computers",
            ],
            "Sports and Fitness": [
                "Cardio Equipment",
                "Indoor/Outdoor Games",
                "Water Sports",
                "Shop By Sport",
                "Camping & Hiking",
                "Fishing"
            ],
            "Other": [
                "Garden",
                "Pet Supplies"
            ]
        }

        cat_keys = list(categories.keys())

        # Extract preprocessor components
        le_category = preprocessor_data['label_encoder_category']
        reference_date = preprocessor_data['reference_date']
        unique_categories = preprocessor_data['unique_categories']
        feature_columns = preprocessor_data['feature_columns']

        # Get data
        category_name = input_data['category']
        future_month = input_data['month']
        avg_price = float(input_data['avg_price'])
        customer_segment = input_data['customer_segment']
        discount_rate = float(input_data['discount_rate'])
        
        # Parse the future date
        future_date = pd.to_datetime(future_month)
        
        # Calculate time features for the future date
        months_since_start = ((future_date - reference_date).days / 30.44)
        
        # Create test data with numerical time features
        test_data = {
            'Category Name': category_name,
            'Average Product Price': avg_price,
            'Customer Segment': customer_segment,
            'Order Item Discount Rate': discount_rate,
            # Time features (numerical - can handle ANY future date!)
            'Year': future_date.year,
            'Month': future_date.month,
            'Quarter': future_date.quarter,
            'Months_Since_Start': int(months_since_start),
            'Month_Sin': np.sin(2 * np.pi * future_date.month / 12),
            'Month_Cos': np.cos(2 * np.pi * future_date.month / 12),
            'Year_Trend': future_date.year - reference_date.year
        }
        
        # Create DataFrame
        test_df = pd.DataFrame([test_data])
        
        # Handle unknown category
        if category_name not in cat_keys:
            print(f"Unknown category '{category_name}' - using default: {cat_keys[0]}")
            test_df['Category Name'] = cat_keys[0]
            category_name = cat_keys[0]
        
        # One-hot encode customer segment
        test_df = pd.get_dummies(test_df, columns=['Customer Segment'], drop_first=True)
        
        # Ensure same columns as training (crucial!)
        test_df = test_df.reindex(columns=feature_columns, fill_value=0)
        
        # Make prediction
        total = 0
        num = len(categories[category_name])
        for subclass in categories[category_name]:
            test_df['Category Name'] = subclass
            test_df['Category Name'] = le_category.transform(test_df['Category Name'])
            total += demand_forecast_model.predict(test_df)
        
        avg_demand = total / num
        
        return avg_demand

    def detect_anomalies(inventory_list):
        anomalies_detected = []
        for item in inventory_list:
            current_location = item['Location']
            predicted_location = Supervised_Models.predict_location({
                "Priority": item['Priority'],
                "Product_Type": item['Category'],
                "Size": item['Size'],
                "Order_Quantity": item['Quantity'],
                "Weight": item['Weight']
            })[0]

            #print(f"\nCurrent Location: {current_location}")
            #print(f"Predicted Location: {predicted_location}")
            if current_location != predicted_location:
                #print(f"Anomaly detected! Item id:{item['ItemId']} is stored at location {current_location} while it should be stored at {predicted_location}.")
                anomalies_detected.append({'ItemId': item['ItemId'], 'ItemName': item['ItemName'], 'CurrentLocation': current_location, 'PredictedLocation': predicted_location})

        return anomalies_detected

s = Supervised_Models

# Testing

print(s.predict_location({
    "Priority": "Medium",
    "Product_Type": "Sports and Fitness",
    "Size": "Medium",
    "Order_Quantity": 12,
    "Weight": 10.78
}))

print(s.detect_anomalies(inventoryData))

result = s.demand_forecast_preprocessor(orderData, inventoryData)
result

print(s.predict_demand_forecast({
    'category': "Clothing",
    'month': "2025-05",
    'avg_price': 10.0,
    'customer_segment': "Consumer",
    'discount_rate': 0.12
}))

# Sections

## Products Overview

In [ ]:
# Product Overview: This section summarizes the overall performance of products
product_overview = client.models.generate_content(
    model=MODEL_ID,
    contents='''
Generate an overview of products based on the following:

1. **Top Products**: Identify the top-selling products by sales volume and revenue over the past month.
2. **Product Performance**: Analyze the performance of each product category in terms of sales, demand, and revenue.
3. **Sales by Product**: Present a summary of sales for each product, including revenue, volume sold, and average price.

Data:
- Sales Data: {sales_data}
- Product Categories: {product_categories_data}
- Product Sales Volume: {sales_volume_data}
- Product Revenue: {product_revenue_data}
'''
)

display(Markdown(product_overview.text))


## Category Distribution

In [ ]:
# Category Distribution: This section analyzes the distribution of sales across categories
category_distribution = client.models.generate_content(
    model=MODEL_ID,
    contents='''
Generate insights on the distribution of sales across categories based on the following:

1. **Category-wise Sales Volume**: Summarize the total sales volume across all product categories.
2. **Category-wise Revenue**: Display the total revenue generated from each product category over the past month.
3. **Category-wise Performance**: Provide insights on which product categories are performing the best in terms of sales, revenue, and customer demand.

Data:
- Sales Volume by Category: {sales_volume_by_category_data}
- Revenue by Category: {revenue_by_category_data}
- Product Category Performance: {category_performance_data}
'''
)

display(Markdown(category_distribution.text))


## Product Usage Forecast

In [ ]:
# Product Usage Forecast: This section forecasts the future usage of products based on historical sales
product_usage_forecast = client.models.generate_content(
    model=MODEL_ID,
    contents='''
Generate a product usage forecast for the upcoming period based on historical data and trends:

1. **Sales Forecast by Product**: Predict the sales volume for each product in the next quarter/year.
2. **Demand Forecast**: Provide a demand forecast for products based on historical sales, usage probabilities, and seasonal patterns.
3. **Future Stock Levels**: Estimate the required stock levels for each product to meet forecasted demand.

Data:
- Historical Sales Data: {historical_sales_data}
- Product Usage Data: {usage_probabilities}
- Seasonal Patterns: {seasonal_sales_patterns_data}
- Current Stock Levels: {current_inventory_data}
'''
)

display(Markdown(product_usage_forecast.text))


## Sales Insights

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Sales Data
sales_data = orderData

# Sales Predictions
current_date = datetime.now()
next_month_date = current_date + relativedelta(months=1)
next_month_yearmonth = next_month_date.strftime("%Y-%m")

sales_pred_input = s.demand_forecast_preprocessor(orderData, inventoryData)
sales_predictions = []

for index, row in sales_pred_input.iterrows():
    sales_predictions.append({
        'Category Name': row.Category,
        'Customer Segment': row.CustomerSegment,
        'Predicted Demand for next month': s.predict_demand_forecast({
            'category': row.Category,
            'month': next_month_yearmonth,
            'avg_price': row.AveragePrice,
            'customer_segment': row.CustomerSegment,
            'discount_rate': row.AverageDiscount
        })[0]
    })

# Product Categories
product_categories = ["Clothing","Technology","Sports and Fitness","Other"]

# Current Inventory
current_inventory = inventoryData

# Usage Probabilities
usage_probabilities = "Currently empty. Please ignore this section for now."

In [ ]:
# Sales Insights Section
section_sales_insights = client.models.generate_content(
    model=MODEL_ID,
    contents=
f'''Generate a sales insights report that describes information for the following areas:

1. Sales Trends: Summarize sales based on the provided data. Provide insights on which product categories are seeing the highest demand.
2. Product Performance: Analyze the best-selling product categories by quantity. Highlight the top 3 performing categories.
3. Product Demand Forecast: Based on the historical sales and usage probability, forecast the demand for the next month.
4. Restocking or Discontinuation: Recommend which products should be restocked and which should be discontinued, based on sales trends and inventory levels.

Data:
- Historical sales data: {sales_data}
- Sales volume predictions for next month: {sales_predictions}
- Product categories: {product_categories}
- Current inventory levels: {current_inventory}
- Usage probabilities: {usage_probabilities}'''
)

display(Markdown(section_sales_insights.text))

## Storage Optimizations

In [ ]:
location_predictions = []
for item in inventoryData:
    location_predictions.append({
        'Item Id': item['ItemId'],
        'Current Location': item['Location'],
        'Predicted Location': s.predict_location({
            'Priority': item['Priority'],
            'Product_Type': item['Category'],
            'Size': item['Size'],
            'Order_Quantity': item['Quantity'],
            'Weight': item['Weight']
        })[0]
    })

section_storage_optimizations = client.models.generate_content(
    model=MODEL_ID,
    contents=
f'''Provide detailed storage optimization recommendations based on:

1. Current storage utilization metrics
2. Model-predicted optimal locations vs current locations
3. List of items flagged for relocation, including:
   - Current location
   - Recommended location

Data:
{inventoryData}
{location_predictions}
'''
)

display(Markdown(section_storage_optimizations.text))

## Anomalies Detected

In [ ]:
section_anomalies_detected = client.models.generate_content(
    model=MODEL_ID,
    contents=
f'''Generate an anomalies section that lists all detected storage anomalies detected in a table. Include each item's current location, predicted location, item id, and name.
Include the reason for each anomaly.

Data:
{s.detect_anomalies(inventoryData)}
'''
)

display(Markdown(section_anomalies_detected.text))

## Summary

In [ ]:
# Read Markdown content
monthly_report = f'''<h1 style="text-align:center;">Monthly Report</h1><br>

# Products Overview:
section_products_overview.text

# Category Distribution:
section_category_distribution.text

# Product Usage Forecast:
section_product_usage.text

# Sales Insights:
{section_sales_insights.text}

# Storage Optimizations:
{section_storage_optimizations.text}

# Anomalies Detected:
{section_anomalies_detected.text}'''

In [ ]:
section_summary = client.models.generate_content(
    model=MODEL_ID,
    contents=
f'''Provide a brief and concise summary of the provided report, covering all the key points highlighted by each section. Highlight the important information for each section in a bullet list, with the final paragraph providing general insight on overall performance.

Report:
{monthly_report}'''
)

display(Markdown(section_summary.text))

# PDF File Generation

In [ ]:
current_date = datetime.now()
monthly_report = f"""# <h1 style="text-align:center;">Monthly Report ({current_date.date()})</h1><br>

# Products Overview:
section_products_overview.text

# Category Distribution:
section_category_distribution.text

# Product Usage Forecast:
section_product_usage.text

# Sales Insights:
{section_sales_insights.text}

# Storage Optimizations:
{section_storage_optimizations.text}

# Anomalies Detected:
{section_anomalies_detected.text}

# Summary:
{section_summary.text}"""

In [ ]:
from markdown_pdf import MarkdownPdf, Section

pdf = MarkdownPdf(toc_level=1)
pdf.add_section(Section(monthly_report)) # Add Section(md_content, user_css=css_content) for custom CSS
pdf.save(f"MonthlyReport_({current_date.date()}).pdf")